In [1]:
# !pip3 install torch
# !pip3 install torchvision
# !pip3 install matplotlib
# !pip3 install torchmetrics

import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
print("Success")

Success


## Sigmoid 문제점
- 작거나, 큰 영역에서 0에 가까운 작은 값이 나온다. 
- Backpropagaion시 Activation 함수에서 Gradient를 곱하게 되는데 이떄 Gradient가 0에 가까우면 Gradient가 소멸 되게 된다. => Gradient Vanishing이라 함

## ReLU
- f(x) = max(0, x)
- x가 0보다 큰 영역에서는 gradient가 1이기 때문에 vanishing 문제가 없다.
- 0보다 작은 영역에서는 Gradinet가 1이기 떄문에 Vanishing이 나타나지만 사용하기에는 문제 없다.

## Weight Initialization
- 0으로 초기화 하면 모든 Gradient가 0이 되서 학습이 되지 않는다.

#### Xavier
- 평균이 0, 표준편차가 : V 로 Weight를 초기화 한다.
- v = sqrt((2 / (input개수 + ouput 개수)))

#### He
- 평균이 0, 표준편차가 : V 로 Weight를 초기화 한다.
- v = sqrt((2 / input개수))



In [9]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(786, 256)
    self.linear2 = nn.Linear(256, 256)
    self.linear3 = nn.Linear(256, 10)
    self._weight_init()

  def _weight_init(self):
    nn.init.xavier_normal_(self.linear1.weight)
    nn.init.xavier_normal_(self.linear2.weight)
    nn.init.xavier_normal_(self.linear3.weight)

model = Network()

print(model.linear1.weight)

Parameter containing:
tensor([[-0.0144, -0.0651, -0.0699,  ...,  0.0174, -0.0781, -0.0704],
        [ 0.0519,  0.0425, -0.0111,  ..., -0.0017,  0.0058,  0.0006],
        [-0.0818, -0.0162,  0.0302,  ...,  0.0191,  0.0877,  0.0150],
        ...,
        [-0.0730,  0.0114, -0.0425,  ..., -0.0184,  0.0456, -0.0233],
        [ 0.1760,  0.0049, -0.0735,  ...,  0.0304,  0.0310,  0.0458],
        [-0.0043,  0.0074, -0.0394,  ...,  0.0100,  0.0594,  0.0064]],
       requires_grad=True)


## Overfitting
- Train 데이터 셋은 높은 정확도를 가지나, Test 데이터 셋(일반화 상황)에서는 낮은 정확도를 갖는 경우

## Dropout
- 학습시 각 레이어의 노드를 무작위로 설정된 비율만큼 학습을 시키지 않는 것
- overfitting을 방지 할 수 있다.
- 매번 다른 네트워크를 사용하여 학습 한다고 생각하여 네트워크 앙상블 효과를 얻을 수 있다. 
- 추론시에는 학습된 만큼 노드에 가중치를 줘서 학습이 많이된 노드에 가중치를 높인다.

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.p = 0.8
    self.layer = nn.Sequential(
        nn.Linear(789, 256),
        nn.ReLU(),
        nn.Dropout(self.p),
        nn.Linear(789, 256),
        nn.ReLU(),
        nn.Dropout(self.p),
        nn.Linear(256, 10),
        nn.Softmax()
    )

    self._weight_init()



model = Network()

## Gradient Vanishing
- Backpropagation 시 Gradient가 0에 가까워져 앞단 Layer에 Gradinet가 전달되지 않는 문제
- activation function을 sigmoid나 계단 함수를 사용하지 않고 relu나 tanh르 사용한다.
- 가중치 초기화 기법을 통해 weight가 0이나 1로 쏠리는 현상을 방지 한다. 

## Gradinet Exploding
- Gradinet가 너무 커져 앞단으로 갈 수록 Weight가 너무크게 갱신된다.(발산)
- learning rate를 줄여 발산 현상을 감소 시킨다.

## Batch Normalization
- Gradinet 문제 해결, 학습 과정이 안정적으로 된다, 학습 속도가 향상 된다.
- 첫 8개에 대한 Batch normalization과 첫8개중 n개만 바뀐 데이터의 Batch normalization의 결과가 다를 수 있다는 문제 점이 있다.
- 따라서, 추론시 batch가 달랐을때 같은 데이터를 다르게 에측하는 문제가 있을 수 있다.
- 학습시 사용된 mean과 variance를 따로 저장 해 두고 추론시에는 저장해둔 mean 과 variance의 평균을 사용 한다. 즉 추론시에는 mean과 variance를 고정하고 추론한다.
따라서 Batch-size가 변화해도 mean, variance가 변경되는 문제가 사라진다.

#### Internal Covariate Shift
- 원래 Input에 데이터 분포에 비해 Layer를 통과 하면서 데이터 분포가 너무 달라지는 문제
- 이 데이터 분포가 달라지는 문제를 Batch Normalization을 사용한다. 
- 각 Layer의 연산이 끝난 후 데이터를 Normalization 한다.
- 첫 입력만 Normalization 하는 것이 아닌 Layer마다 Internal Covariate Shift 문제가 발생하기 때문에 각 Layer마다 Normalization 을 한다.